In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as tts,GridSearchCV
import statsmodels.api as sm
from sklearn.linear_model import Lasso, LassoCV, ElasticNet, ElasticNetCV,LinearRegression,BayesianRidge,Ridge, LogisticRegression
from sklearn.svm import SVR
import sklearn.metrics as metrics
from sklearn.neighbors import KNeighborsRegressor as KNNR
import scipy
import plotly.plotly as py
import plotly.graph_objs as go
import pygsheets

%matplotlib inline
pd.options.display.max_rows=100
pd.options.display.max_columns=100

- Ideas
    - Bring back data
    - try cleaning it more thoroughly
    - try creating a few more interesting features (look at their distributions!!)
    - Try an MCMC or another approach
    - try PCA then a regression
- Timeline
    - do more extensive data cleaning during the week
    - convert models to DAG

In [31]:
gc = pygsheets.authorize(outh_file='/Users/irarickman/client_secret.json')
file = 'Activity data'
sh = gc.open(file)

wks_1 = sh[0]
strava=wks_1.get_as_df(empty_value=np.nan)
strava.dropna(how='all',inplace=True)

wks_2=sh[1]
whoop=wks_2.get_as_df(empty_value=np.nan)
whoop.dropna(how='all',inplace=True)
whoop.dropna(axis=1,how='all',inplace=True)

wks_3=sh[2]
acts=wks_3.get_as_df(empty_value=np.nan)
acts.dropna(how='all',inplace=True)
acts.dropna(axis=1,how='all',inplace=True)


In [32]:
print(strava.shape)
strava.columns

(276, 44)


Index(['achievement_count', 'athlete_count', 'average_speed', 'average_watts',
       'comment_count', 'commute', 'device_watts', 'distance', 'elapsed_time',
       'elev_high', 'elev_low', 'external_id', 'flagged', 'gear_id',
       'has_kudoed', 'id', 'kilojoules', 'kudos_count', 'manual', 'max_speed',
       'max_watts', 'moving_time', 'name', 'photo_count', 'private',
       'start_date', 'start_date_local', 'timezone', 'total_elevation_gain',
       'total_photo_count', 'trainer', 'type', 'upload_id',
       'weighted_average_watts', 'workout_type', 'miles', 'race',
       'date_string', 'moving_minutes', 'elapsed_minutes', 'rest', 'avg_mph',
       'time_since_last_act', 'order'],
      dtype='object')

In [33]:
strava.type.value_counts()

Run       209
Ride       38
Swim       27
Walk        1
Rowing      1
Name: type, dtype: int64

In [34]:
strava[strava.type=='Run'].describe()

,achievement_count,athlete_count,average_speed,average_watts,comment_count,distance,elapsed_time,elev_high,elev_low,gear_id,id,kilojoules,kudos_count,max_speed,max_watts,moving_time,photo_count,total_elevation_gain,total_photo_count,upload_id,weighted_average_watts,workout_type,miles,race,moving_minutes,elapsed_minutes,rest,avg_mph,time_since_last_act,order
count,209.000000,209.000000,209.000000,0.0,209.000000,209.000000,209.000000,209.000000,209.000000,0.0,2.090000e+02,0.0,209.000000,209.000000,0.0,209.000000,209.0,209.000000,209.000000,2.090000e+02,0.0,13.000000,209.000000,209.000000,209.000000,209.000000,209.000000,209.000000,208.000000,209.000000
mean,1.191388,3.598086,3.163263,NaN,0.033493,8997.214833,3256.799043,103.333971,67.201914,NaN,1.994844e+09,NaN,3.497608,6.542584,NaN,2849.813397,0.0,76.263158,0.023923,2.131702e+09,NaN,0.692308,5.590608,0.043062,47.496890,54.279984,6.783094,7.076220,-40.692308,1.114833
std,2.434077,17.324427,0.361951,NaN,0.227503,4040.303292,1397.756921,181.152224,137.428923,NaN,2.636118e+08,NaN,2.603851,2.442129,NaN,1281.460243,0.0,76.304001,0.228713,2.740424e+08,NaN,0.480384,2.510527,0.203485,21.357671,23.295949,8.417074,0.809684,23.607940,0.334290
min,0.000000,1.000000,0.802000,NaN,0.000000,1705.200000,786.000000,-100.200000,-246.800000,NaN,1.483968e+09,NaN,0.000000,3.400000,NaN,439.000000,0.0,8.000000,0.000000,1.598846e+09,NaN,0.000000,1.059562,0.000000,7.316667,13.100000,0.000000,1.794074,-171.000000,1.000000
25%,0.000000,1.000000,3.047000,NaN,0.000000,5957.400000,2224.000000,60.000000,19.400000,NaN,1.781150e+09,NaN,2.000000,4.800000,NaN,1963.000000,0.0,40.000000,0.000000,1.911685e+09,NaN,0.000000,3.701756,0.000000,32.716667,37.066667,2.200000,6.816139,-52.000000,1.000000
50%,0.000000,1.000000,3.178000,NaN,0.000000,8644.000000,3055.000000,67.600000,46.200000,NaN,2.036574e+09,NaN,3.000000,5.900000,NaN,2723.000000,0.0,56.000000,0.000000,2.176231e+09,NaN,1.000000,5.371131,0.000000,45.383333,50.916667,3.566667,7.109186,-37.000000,1.000000
75%,1.000000,1.000000,3.322000,NaN,0.000000,10715.700000,4086.000000,89.000000,54.800000,NaN,2.214673e+09,NaN,5.000000,7.500000,NaN,3547.000000,0.0,81.000000,0.000000,2.359098e+09,NaN,1.000000,6.658425,0.000000,59.116667,68.100000,6.783333,7.431314,-25.000000,1.000000
max,13.000000,181.000000,4.304000,NaN,2.000000,22539.300000,8744.000000,1998.400000,1418.800000,NaN,2.393119e+09,NaN,17.000000,17.400000,NaN,7364.000000,0.0,611.000000,3.000000,2.544668e+09,NaN,1.000000,14.005267,1.000000,122.733333,145.733333,43.100000,9.628048,-1.000000,3.000000


In [39]:
multi_runs.dtypes

achievement_count         float64
athlete_count             float64
average_speed             float64
average_watts             float64
comment_count             float64
commute                    object
device_watts               object
distance                  float64
elapsed_time              float64
elev_high                 float64
elev_low                  float64
external_id                object
flagged                    object
gear_id                   float64
has_kudoed                 object
id                        float64
kilojoules                float64
kudos_count               float64
manual                     object
max_speed                 float64
max_watts                 float64
moving_time               float64
name                       object
photo_count               float64
private                    object
start_date                 object
start_date_local           object
timezone                   object
total_elevation_gain      float64
total_photo_co

In [51]:
## since there are some times where i accidentally stored a run as two separate activities, 
## trying to find them to see if it's worth it to join them
strava['multi_act']=strava.groupby('date_string').id.transform(len)>1
strava['multi_run']=strava.groupby(['date_string','type']).id.transform(len)>1
multi_runs=strava[(strava.multi_run)&(strava.type=='Run')]
multi_runs['start_date_time']=pd.to_datetime(multi_runs.start_date_local)
# multi_runs['end_time']=multi_runs.start_date_local+multi_runs.elapsed_time/60.0
# multi_runs.end_time
multi_runs['end_time']=multi_runs.start_date_time + pd.to_timedelta(multi_runs.elapsed_time,unit='s')
multi_runs[['start_date_local','end_time']]

/Users/irarickman/anaconda2/envs/py35/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/Users/irarickman/anaconda2/envs/py35/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,start_date_local,end_time
6,2019-05-15 05:59:32+00:00,2019-05-15 06:34:19
7,2019-05-15 06:36:58+00:00,2019-05-15 07:49:42
89,2018-07-15 06:50:01+00:00,2018-07-15 07:36:18
91,2018-07-15 08:57:52+00:00,2018-07-15 09:50:52
148,2018-10-03 06:27:25+00:00,2018-10-03 07:00:13
149,2018-10-03 07:20:42+00:00,2018-10-03 07:45:46
185,2018-12-25 16:51:37+00:00,2018-12-25 17:21:01
186,2018-12-26 06:40:44+00:00,2018-12-26 07:05:46
192,2019-01-05 12:38:09+00:00,2019-01-05 13:34:22
193,2019-01-05 13:40:22+00:00,2019-01-05 13:53:35


In [ ]:
examine=['2019-01-05','2019-01-17','2019-01-1']
strava[strava.date_string in ]

In [17]:
## to simplify things, I want to delete the days with multiple activities
strava['multi_act']=strava.groupby('date_string').id.transform(len)>1
single_acts=strava[(strava.multi_act==False)&(strava.type=='Run')]
len(single_acts)

169